# Setup


In [ ]:
import ee
import urllib.request
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import PIL
from PIL import Image, ImageDraw
from osgeo import gdal
from osgeo import osr
import numpy as np
import os, sys
import random
import shutil
import pandas as pd

ee.Authenticate()
ee.Initialize()

In [55]:
# export settings
export_path = "single_export"
name="moroccosolarpark"

x=-6.869816026689191
y=31.034143640821156

In [56]:
# Variety of Configuations
vis_min = 0  #Visualization settings for the thumbnail
vis_max = 1024 #Visualization settings for the thumbnail
vis_bands = ['B4', 'B3', 'B2'] #Includes the bands for RGB
imageDimensions = '512x512' #Set thumbnail image size (can't be too big, or you run into problems)
nir_bands = ['B8'] #Includes the bands for nir
swirB11_bands = ['B11'] #Includes the bands for swir B11
swirB12_bands = ['B12'] #Includes the bands for swir B11

#Choose your location to request an image from
longitude = x
latitude = y

center = ee.Geometry.Point(longitude,latitude)
# Import Sentinel dataset
s2 = (ee.ImageCollection("COPERNICUS/S2_SR")
  .filterBounds(center)
  .sort('CLOUDY_PIXEL_PERCENTAGE')
  .filterDate('2022-01-01', '2023-12-30')
  .first()
)

try:
  global sentinel_footprint
  sentinel_footprint = (ee.Geometry.Polygon((s2.getInfo().get('properties').get('system:footprint').get('coordinates'))))
except:
  print("No Footprint found")

footprint = (ee.Geometry.Polygon((s2.getInfo().get('properties').get('system:footprint').get('coordinates'))))

sentinel_centroid = center
# Create a rectangular export area
# standard buffer: 2530
exportAreaSmall = (sentinel_centroid.buffer(2530).bounds())

if sentinel_footprint.contains(exportAreaSmall,1).getInfo() == True:
  exportArea = exportAreaSmall
else:
  print("No fit")

# save rgb
s2Vis = {
    'region': exportArea,
    'crs': (s2.select('B4').projection()),
    'dimensions': imageDimensions,
    'format': 'jpg',
    'min': vis_min,
    'max': vis_max,
    'bands': vis_bands
}

s2_url = (s2.getThumbURL(s2Vis))

#Change the location where the images are saved by replacing "content" with the location in your Google Drive
s2_name = "{}/{}.jpg".format(export_path,name)
# save
urllib.request.urlretrieve(s2_url, s2_name)

# find min and max of df
x_min = min(pd.DataFrame(exportArea.getInfo().get('coordinates')[0], columns = ['x', 'y'])['x'])
y_min = min(pd.DataFrame(exportArea.getInfo().get('coordinates')[0], columns = ['x', 'y'])['y'])
x_max = max(pd.DataFrame(exportArea.getInfo().get('coordinates')[0], columns = ['x', 'y'])['x'])
y_max = max(pd.DataFrame(exportArea.getInfo().get('coordinates')[0], columns = ['x', 'y'])['y'])

# generate a tif file with all bands!
_region = ee.Geometry.BBox(x_min, y_min, x_max, y_max)
task = ee.batch.Export.image.toDrive(
    image = s2.select('B2','B3','B4','B8','B11','B12'),
    description=name,
    folder='single_export',
    region=_region,
    dimensions=imageDimensions,
    crs=(s2.select('B4').projection())
)
task.start()


In [50]:
observation_dates = [
    ('2019-01-01', '2019-01-30'),
    ('2019-02-01', '2019-02-30'),
    ('2019-03-01', '2019-03-30'),
    ('2019-04-01', '2019-04-30'),
    ('2019-05-01', '2019-05-30'),
    ('2019-06-01', '2019-06-30'),
    ('2019-07-01', '2019-07-30'),
    ('2019-08-01', '2019-08-30'),
    ('2019-09-01', '2019-09-30'),
    ('2019-10-01', '2019-10-30'),
    ('2019-11-01', '2019-11-30'),
    ('2019-12-01', '2019-12-30'),
    ('2020-01-01', '2020-01-30'),
    ('2020-02-01', '2020-02-30'),
    ('2020-03-01', '2020-03-30'),
    ('2020-04-01', '2020-04-30'),
    ('2020-05-01', '2020-05-30'),
    ('2020-06-01', '2020-06-30'),
    ('2020-07-01', '2020-07-30'),
    ('2020-08-01', '2020-08-30'),
    ('2020-09-01', '2020-09-30'),
    ('2020-10-01', '2020-10-30'),
    ('2020-11-01', '2020-11-30'),
    ('2020-12-01', '2020-12-30'),
    ('2021-01-01', '2021-01-30'),
    ('2021-02-01', '2021-02-30'),
    ('2021-03-01', '2021-03-30'),
    ('2021-04-01', '2021-04-30'),
    ('2021-05-01', '2021-05-30'),
    ('2021-06-01', '2021-06-30'),
    ('2021-07-01', '2021-07-30'),
    ('2021-08-01', '2021-08-30'),
    ('2021-09-01', '2021-09-30'),
    ('2021-10-01', '2021-10-30'),
    ('2021-11-01', '2021-11-30'),
    ('2021-12-01', '2021-12-30'),
    ('2022-01-01', '2022-01-30'),
    ('2022-02-01', '2022-02-30'),
    ('2022-03-01', '2022-03-30'),
    ('2022-04-01', '2022-04-30'),
    ('2022-05-01', '2022-05-30'),
    ('2022-06-01', '2022-06-30'),
    ('2022-07-01', '2022-07-30'),
    ('2022-08-01', '2022-08-30'),
    ('2022-09-01', '2022-09-30'),
    ('2022-10-01', '2022-10-30'),
    ('2022-11-01', '2022-11-30'),
    ('2022-12-01', '2022-12-30')
    ]


In [51]:
# Variety of Configuations
vis_min = 0  #Visualization settings for the thumbnail
vis_max = 1024 #Visualization settings for the thumbnail
vis_bands = ['B4', 'B3', 'B2'] #Includes the bands for RGB
imageDimensions = '512x512' #Set thumbnail image size (can't be too big, or you run into problems)
nir_bands = ['B8'] #Includes the bands for nir
swirB11_bands = ['B11'] #Includes the bands for swir B11
swirB12_bands = ['B12'] #Includes the bands for swir B11

#Choose your location to request an image from
longitude = x
latitude = y
for start,end in observation_dates:
  center = ee.Geometry.Point(longitude,latitude)
  # Import Sentinel dataset
  s2 = (ee.ImageCollection("COPERNICUS/S2_SR")
    .filterBounds(center)
    .sort('CLOUDY_PIXEL_PERCENTAGE')
    .filterDate(start, end)
    .first()
  )
  # Working, but with getInfo(), which makes the code a bit slower
  try:
    global sentinel_footprint
    sentinel_footprint = (ee.Geometry.Polygon((s2.getInfo().get('properties').get('system:footprint').get('coordinates'))))
  except:
    print("No Footprint found")
    continue

  footprint = (ee.Geometry.Polygon((s2.getInfo().get('properties').get('system:footprint').get('coordinates'))))

  sentinel_centroid = center
  # Create a rectangular export area
  # standard buffer: 2530
  exportAreaSmall = (sentinel_centroid.buffer(2530).bounds())

  if sentinel_footprint.contains(exportAreaSmall,1).getInfo() == True:
    exportArea = exportAreaSmall
  else:
    print("No fit")

  # save rgb
  s2Vis = {
      'region': exportArea,
      'crs': (s2.select('B4').projection()),
      'dimensions': imageDimensions,
      'format': 'jpg',
      'min': vis_min,
      'max': vis_max,
      'bands': vis_bands
  }

  s2_url = (s2.getThumbURL(s2Vis))

  #Change the location where the images are saved by replacing "content" with the location in your Google Drive
  s2_name = "{}/{}-{}-{}.jpg".format(export_path,name,start,end)
  # save
  urllib.request.urlretrieve(s2_url, s2_name)

  # find min and max of df
  x_min = min(pd.DataFrame(exportArea.getInfo().get('coordinates')[0], columns = ['x', 'y'])['x'])
  y_min = min(pd.DataFrame(exportArea.getInfo().get('coordinates')[0], columns = ['x', 'y'])['y'])
  x_max = max(pd.DataFrame(exportArea.getInfo().get('coordinates')[0], columns = ['x', 'y'])['x'])
  y_max = max(pd.DataFrame(exportArea.getInfo().get('coordinates')[0], columns = ['x', 'y'])['y'])

  # generate a tif file with all bands!
  _region = ee.Geometry.BBox(x_min, y_min, x_max, y_max)
  task = ee.batch.Export.image.toDrive(
      image = s2.select('B2','B3','B4','B8','B11','B12'),
      description="{}-{}-{}.jpg".format(name,start,end),
      folder='single_export',
      region=_region,
      dimensions=imageDimensions,
      crs=(s2.select('B4').projection())
  )
  task.start()


No Footprint found
No Footprint found
No Footprint found
No Footprint found


In [ ]:
import os
import sys
import subprocess
import glob
import string
from osgeo import gdal

print('Merging the DEMM files...')

# Find all files with a .tif extension
input_files = glob.glob("/Users/henrikkother/Desktop/Master-Thesis KIT/Solar_panels/code/interference_data/marokko2023/S2*.tif")

# Define the output file
output_file = "/Users/henrikkother/Desktop/Master-Thesis KIT/Solar_panels/code/interference_data/geotiff.tiff"

# Create a dictionary to store the original file names as keys and new short file names as values
original_names = {}


#Build VRT from input files 
vrt_file = "merged.vrt"
gdal.BuildVRT(vrt_file,input_files)

# Translate VRT to TIFF
gdal.Translate(output_file,vrt_file)

# remove the vrt file
os.remove(vrt_file)